Need to upload:
*   shape_predictor_81_face_landmarks.dat
*   mask_detect.h5
*   videos






In [ ]:
import cv2
import os
import zipfile
import shutil
from sklearn.model_selection import train_test_split
from google.colab import files
import requests
from google.colab.patches import cv2_imshow


Face detection using Haar Cascades

In [ ]:
def face_detector(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_classifier = cv2.CascadeClassifier(os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_default.xml'))
    scale = 4.0
    faces = face_classifier.detectMultiScale(gray_img, scale, 5)
    # print("LEN: ", len(faces))

    if len(faces) > 0:
      for (x, y, w, h) in faces:
          roi = img[y:y+h, x:x+w]
          return roi
    if len(faces) <= 0:
      scale = scale - 0.1
      while scale > 1.1:
        # print("Nie ma twarzy, scala: ", scale)
        faces = face_classifier.detectMultiScale(gray_img, scale, 5)
        if len(faces) > 0:
          for (x, y, w, h) in faces:
              roi = img[y:y+h, x:x+w]
              return roi
        scale = scale - 0.1


    return None


Extracting the upper facial region when the lower part is obscured by a mask

In [ ]:
predictor = dlib.shape_predictor("shape_predictor_81_face_landmarks.dat")

def extract_forehead_face(img):
    x,y =img.shape[0], img.shape[1]
    dlib_rect = dlib.rectangle(1,1,x-1,y-1)
    landmarks = predictor(img, dlib_rect)

    forehead_face_roi = np.array([
        (landmarks.part(20).x, landmarks.part(20).y),  # Left brow
        (landmarks.part(25).x, landmarks.part(25).y),
        (landmarks.part(70).x, landmarks.part(70).y),  # Right brow
        (landmarks.part(71).x, landmarks.part(71).y),
        (landmarks.part(1).x, landmarks.part(1).y),    # Left side
        (landmarks.part(17).x, landmarks.part(17).y),  # Right side
        (landmarks.part(16).x, landmarks.part(16).y),
        (landmarks.part(6).x, landmarks.part(16).y),
        (landmarks.part(42).x, landmarks.part(42).y),  # Left eye bottom
        (landmarks.part(47).x, landmarks.part(47).y),  # Right eye bottom
    ], dtype=np.int32)

    x, y, w, h = cv2.boundingRect(forehead_face_roi)
    new_y = max(0, int(y - 0.2 * h))
    new_h = int(h + 0.1 * h)
    forehead_face = img[new_y:new_y + new_h, x:x + w]

    return forehead_face




Detecting a mask and extracting upper part of the face


In [ ]:
loaded_model = load_model("mask_detect.h5")

def mask_detector(roi_img):
    resized_roi_img = cv2.resize(roi_img, (224, 224))
    normalized_roi_img = resized_roi_img / 255.0
    img_for_prediction = normalized_roi_img.reshape(1, 224, 224, 3)
    mask_prediction = loaded_model.predict(img_for_prediction)
    result = np.argmax(mask_prediction)
    if result == 0:
        # cv2.putText(img, 'Mask Detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        upper_face_roi = extract_forehead_face(roi_img)

        return upper_face_roi

    else:
        # cv2.putText(img, 'No Mask Detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        print("no")
        return roi_img


Extracting frames from videos, splitting into test and train data, saving into zip files

In [ ]:
def extract_frames_and_process(video_paths, local_folder, output_folder, zip_filename):
    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    image_id = 1

    for vid in video_paths:
        success = True
        count = 1

        video = cv2.VideoCapture(vid)

        while success:
            success, frame = video.read()

            if success == True:
                if count % 1 == 0:
                    name = f'{local_folder}/{image_id}.jpg'
                    cv2.imwrite(name, frame)
                    image_id += 1
                count += 1
            else:
                break

    print("Total Extracted Frames:", image_id)
    print("Frames saved to:", local_folder)

    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(local_folder):
        if filename.endswith(".jpg"):
            img_path = os.path.join(local_folder, filename)

            test_img = cv2.imread(img_path)


            roi = face_detector(test_img)
            roi = face_detector(test_img)
            if roi is not None:
                pred = mask_detector(roi)

                if pred is not None and np.any(pred):

                    output_path = os.path.join(output_folder, f"{filename}")
                    cv2.imwrite(output_path, pred)
                else:
                    print(f"Empty or invalid pred for image: {img_path}")

    print("Total images processed:", len(os.listdir(output_folder)))


    with zipfile.ZipFile(zip_filename, 'w') as zip_ref:

        for filename in os.listdir(output_folder):
            if filename.endswith(".jpg"):
                img_path = os.path.join(output_folder, filename)
                zip_ref.write(img_path, arcname=filename)

    print(f"Images in {output_folder} saved to {zip_filename}")

    files.download(zip_filename)

In [ ]:
vid_paths=["SYLWIA_1.mp4","SYLWIA_2.mp4","SYLWIA_3.mp4","SYLWIA_4.mp4"]
extract_frames_and_process(vid_paths,"Syl","Syl_o","syl_pred.zip")

In [ ]:
data_path = '/content/data/Dataset'

classes = ["Emilka", "Julka", "Kacper", "Kaja", "Karolina", "Kuba", "Maciek", "Madzia", "Ola", "Sylwia", "Szczepan"]

test_size = 0.2
random_seed = 42

train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

for class_name in classes:
    class_data_path = os.path.join(data_path, class_name)

    files = os.listdir(class_data_path)

    train_files, test_files = train_test_split(files, test_size=test_size, random_state=random_seed)

    class_train_path = os.path.join(train_path, class_name)
    class_test_path = os.path.join(test_path, class_name)

    os.makedirs(class_train_path, exist_ok=True)
    os.makedirs(class_test_path, exist_ok=True)

    for file in train_files:
        shutil.move(os.path.join(class_data_path, file), os.path.join(class_train_path, file))

    for file in test_files:
        shutil.move(os.path.join(class_data_path, file), os.path.join(class_test_path, file))

sample_class = classes[0]
print(f"Number of training samples for {sample_class}: {len(os.listdir(os.path.join(train_path, sample_class)))}")
print(f"Number of testing samples for {sample_class}: {len(os.listdir(os.path.join(test_path, sample_class)))}")


In [ ]:
def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)


def download_zip(local_filename):
    with open(local_filename, 'wb') as file:
        file.write(response.content)


In [ ]:
folder_to_zip1 = '/content/data/Dataset/train'
zip_file_path1 = '/content/train.zip'
folder_to_zip2 = '/content/data/Dataset/test'
zip_file_path2 = '/content/test.zip'

zip_folder(folder_to_zip1, zip_file_path1)
zip_folder(folder_to_zip2, zip_file_path2)

In [ ]:
local_zip_file1 = '/downloads/train.zip'
local_zip_file2 = '/downloads/test.zip'

download_zip(local_zip_file1)
download_zip(local_zip_file2)